In [1]:
#参考https://github.com/NZbryan/NLP_bert
##去https://huggingface.co/bert-base-chinese 下载tf_model.h5到./pre_model/
import logging
logging.basicConfig(level=logging.ERROR)
from transformers import TFBertForSequenceClassification,BertTokenizer
import tensorflow as tf

import pandas as pd
from sklearn.model_selection import train_test_split

import sys 
sys.path.append("..") 
from utils import data_helper

In [17]:
train_ds='../data/train/labeled_data.csv'
train_df=pd.read_csv(train_ds)[['content','class_label']]
train_df

,content,class_label
0,韩美决定每年在黄海举行反潜联合军演(图)中新网10月19日电 据韩国《朝鲜日报》网站19日报...,时政
1,王鹏：商业地产重要的三个环节董利：刚才蔡总介绍了一下台湾的便利店，其实便利店在改变着人们的生...,房产
2,碧桂园股份掉期亏损净利降66%料今年销售额达190亿 杨国强对楼市表示“审慎乐观” 东方早报...,房产
3,刘新虎：关于丽泽商务区的定位问题【刘新虎】：主要是为了学习，因为以前在丰台区开发过项目，20...,房产
4,回顾2010年的邮市：各品种多版块都很给力老票精品很给力。2010年的春天，老票精品吹响了邮...,财经
...,...,...
6995,英媒刊登照片展示全球正在消融的冰川(图)新浪科技讯 北京时间4月30日消息，据英国《卫报》报...,科技
6996,搭配QA：厚厚羽绒服怎么穿出流行感Q：冬天不得不穿上厚厚的羽绒服御寒，可以大部分羽绒服设计都...,时尚
6997,加以零星别致的摆设(图)快乐来临时，一切都是美好的，清新的色调，加以零星别致的摆设，体现出充...,家居
6998,基地组织训练5岁娃娃兵练习射击充当人弹(图)中新网7月11日电 据英国媒体10日报道，英国一...,时政


In [19]:
#设置截断长度
text_max_length=4000
idxs=train_df.loc[train_df['content'].str.len()>text_max_length].index
#print(test_df.loc[test_df['content'].str.len()>text_max_length].index)
train_df.loc[train_df['content'].str.len()>text_max_length,'content']=train_df.loc[train_df['content'].str.len()>text_max_length,'content'].apply(lambda x:x[:text_max_length].rsplit('。',1)[0])
print(len(train_df[train_df['content'].str.len()>text_max_length]['content'])/len(train_df))
train_df.sample(10)
train_df.shape

0.0


(7000, 2)

In [24]:
import re
#把，。！等换成空格
def replace_marks(line):
    line = str(line)
    if line.strip()=='':
        return ''
    line=re.sub(',|。|！|？|；|，',' ',line)
    return line

In [25]:
train_df['cleaned_content']=train_df['content'].apply(replace_marks)
train_df.sample(10)

,content,class_label,cleaned_content
5565,组图：初涉职场的小妞着装TOP4一只脚刚刚走出校园，另一只脚就买入了职场，这种转变可能会让毫...,时尚,组图：初涉职场的小妞着装TOP4一只脚刚刚走出校园 另一只脚就买入了职场 这种转变可能会让毫...
4437,中国欧盟商会建设工作组主席柳骏勇发表演讲柳骏勇：我代表我们商会非常高兴能够参加今天的会议，能...,房产,中国欧盟商会建设工作组主席柳骏勇发表演讲柳骏勇：我代表我们商会非常高兴能够参加今天的会议 能...
3901,看过来：在日本博士找工作有多难？ 京都大学日前推出新举措：决定派遣尚未就业的博士学位获得者赴...,教育,看过来：在日本博士找工作有多难 京都大学日前推出新举措：决定派遣尚未就业的博士学位获得者赴...
5123,重要公告：参赛报名别盲目 看准要求再行动由新浪网和澳际教育集团强强联手举办的“澳际留美天才挑...,教育,重要公告：参赛报名别盲目 看准要求再行动由新浪网和澳际教育集团强强联手举办的“澳际留美天才挑...
4399,亲历：巴厘岛的SPA好到难以想象(组图)刚一回办公室上班就一大堆事在等着，我的实习生略带哭腔...,教育,亲历：巴厘岛的SPA好到难以想象(组图)刚一回办公室上班就一大堆事在等着 我的实习生略带哭腔...
4886,国货瞄准快消时尚 叫板H&M与ZARA(图)编者按：美特斯-邦威服饰初出道时，竞争对手里需要...,时尚,国货瞄准快消时尚 叫板H&M与ZARA(图)编者按：美特斯-邦威服饰初出道时 竞争对手里需要...
4983,最新趋势：“留学金融”预热出国高峰高考“硝烟未尽”，留学市场已经“烽烟四起”。每年的8、9月...,教育,最新趋势：“留学金融”预热出国高峰高考“硝烟未尽” 留学市场已经“烽烟四起” 每年的8、9月...
4941,科学家设想用充气球登火星找生命(组图)1、充气“侦察兵”被置入火星研究实验室搭载的盒子里。登...,科技,科学家设想用充气球登火星找生命(组图)1、充气“侦察兵”被置入火星研究实验室搭载的盒子里 登...
443,各种材质家具各有所长(图)窍门5 各种材质家具各有所长材料的质地和肌理决定了产品的外观感受。...,家居,各种材质家具各有所长(图)窍门5 各种材质家具各有所长材料的质地和肌理决定了产品的外观感受 ...
4010,解读：美国名校申研成功“关键词”据2009年美国不少综合排名前50的研究生院所公布的数据显示...,教育,解读：美国名校申研成功“关键词”据2009年美国不少综合排名前50的研究生院所公布的数据显示...


In [32]:
cn_class_dic={0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经',7: '游戏',8: '娱乐',9: '体育'}
def getY(line):
    for key in cn_class_dic:
        if(cn_class_dic[key]==line):
            return key
    return -1

In [33]:
train_raw = train_df[['cleaned_content','class_label']]

train_raw.columns=['text','label']
train_raw['y']=train_raw['label'].apply(getY)
train_raw.sample(10)

,text,label,y
6544,存款6000万却贷不到20万 民企急盼小额贷款实例“重庆有头面的大民营企业 有哪家的老总能站...,财经,6
3913,澳大利亚拟整顿教育机构维护留学生权益本报讯 据新华网报道：澳大利亚副总理朱莉娅·吉拉德近日...,教育,2
789,非诚勿扰北海道：秦奋忏悔的小教堂(组图)秦奋忏悔的小教堂－－－基督兄弟团斜里教会影片中秦奋的...,教育,2
5803,六教授联名投诉工程院院士抄袭剽窃(图)本报记者 彭冰日前 本报接到读者来信 反映有6位教授联...,科技,5
397,地下炒汇骗术翻新：借壳商品期货每经记者李正发自北京随着近期国际汇率市场动荡不安 投资者们开始...,财经,6
6639,香港今年68人服用壮阳药感到不适3人死亡中新网9月4日电 据香港明报报道 香港元朗一名中年男...,时政,4
6251,申请英国硕士不能不知道的六大问题英国一直是很多学生读硕的首选 但是由于英国大学数量众多 录取...,教育,2
355,冯小刚 游离于喧闹市区的古朴奢华家(组图)冯小刚的家既不在游离于喧闹市区的豪华别墅的一隅 也...,家居,0
6831,渣打与客户签署标准协议处理投诉唐先生说“一旦谈妥 渣打有一些条件 比如对媒体、监管机构封口 ...,财经,6
6110,2009年教育部批准境外正规高校名单：法国(11) 附录二：法国教育部认可的“巴黎高科...,教育,2


In [4]:
def convert_example_to_feature(review):
  
  # combine step for tokenization, WordPiece vector mapping, adding special tokens as well as truncating reviews longer than the max length
	return tokenizer.encode_plus(review, 
	            add_special_tokens = True, # add [CLS], [SEP]
	            max_length = max_length, # max length of the text that can go to BERT
	            pad_to_max_length = True, # add [PAD] tokens
	            return_attention_mask = True, # add attention mask to not focus on pad tokens
		    truncation=True
	          )
# map to the expected input to TFBertForSequenceClassification, see here 
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_examples(ds, limit=-1):
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    if (limit > 0):
        ds = ds.take(limit)
  
    for index, row in ds.iterrows():
        review = row["text"]
        label = row["y"]
        bert_input = convert_example_to_feature(review)
  
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)



def split_dataset(df):
    train_set, x = train_test_split(df, 
        stratify=df['label'],
        test_size=0.1, 
        random_state=42)
    val_set, test_set = train_test_split(x, 
        stratify=x['label'],
        test_size=0.5, 
        random_state=43)

    return train_set,val_set, test_set


# class TFBertForMultilabelClassification(TFBertPreTrainedModel):

#     def __init__(self, config, *inputs, **kwargs):
#         super(TFBertForMultilabelClassification, self).__init__(config, *inputs, **kwargs)
#         self.num_labels = config.num_labels

#         self.bert = TFBertMainLayer(config, name='bert')
#         self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
#         self.classifier = tf.keras.layers.Dense(config.num_labels,
#                                                 kernel_initializer=get_initializer(config.initializer_range),
#                                                 name='classifier',
#                                                 activation='sigmoid')

#     def call(self, inputs, **kwargs):
#         outputs = self.bert(inputs, **kwargs)

#         pooled_output = outputs[1]

#         pooled_output = self.dropout(pooled_output, training=kwargs.get('training', False))
#         logits = self.classifier(pooled_output)

#         outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

#         return outputs  # logits, (hidden_states), (attentions)


In [36]:


#预处理数据

model_path = './pre_model/bert-base-chinese-vocab.txt' #模型路径，建议预先下载(https://huggingface.co/bert-base-chinese#)

max_length = 256
batch_size = 64
learning_rate = 2e-5
number_of_epochs = 2
num_classes = 10 # 类别数


# split data
train_data,val_data, test_data = split_dataset(train_raw)

In [37]:

# tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)
# train dataset
ds_train_encoded = encode_examples(train_data).shuffle(10000).batch(batch_size)
# val dataset
ds_val_encoded = encode_examples(val_data).batch(batch_size)
# test dataset
ds_test_encoded = encode_examples(test_data).batch(batch_size)



Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
/usr/local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1938: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [38]:
#建立模型
model = TFBertForSequenceClassification.from_pretrained('./pre_model', num_labels=num_classes)

# optimizer Adam recommended
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)
# we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
print(model.summary())

Some layers from the model checkpoint at ./pre_model were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ./pre_model and are newly initialized: ['dropout_75', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  102267648 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  7690      
Total params: 102,275,338
Trainable params: 102,275,338
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#训练模型
bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs, validation_data=ds_val_encoded)
# evaluate test_set
print("# evaluate test_set:",model.evaluate(ds_test_encoded))


Epoch 1/2
 2/99 [..............................] - ETA: 1:17:02 - loss: 2.4475 - accuracy: 0.0625

In [ ]:
#验证数据并输出结果